In [1]:
!pip install pandas pyarrow scikit-learn matplotlib seaborn

In [2]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [3]:
from google.colab import files

uploaded = files.upload()

df = pd.read_parquet('polymarket_data.parquet')

print(f"Dataset shape: {df.shape}")
print(f"Target distribution:\n{df['won'].value_counts()}")

Saving polymarket_data.parquet to polymarket_data.parquet
Dataset shape: (5896882, 13)
Target distribution:
won
0    2948441
1    2948441
Name: count, dtype: int64


In [5]:
df = df.sort_values(['market_id', 'token_id', 'timestamp'])

LOOKBACK = 20
FORECAST = 10

sequences = []
targets = []
market_ids = []

for (market_id, token_id), group in df.groupby(['market_id', 'token_id']):
  prices = group['price'].values
  hours_to_res = group['hours_to_resolution'].values

  for i in range(len(prices) - LOOKBACK - FORECAST + 1):
    seq_prices = prices[i:i+LOOKBACK]
    seq_hours = hours_to_res[i:i+LOOKBACK]

    sequence = np.column_stack([seq_prices, seq_hours])
    target = prices[i + LOOKBACK: i + LOOKBACK + FORECAST]

    sequences.append(sequence)
    targets.append(target)
    market_ids.append(market_id)

X = np.array(sequences)
y = np.array(targets)
market_ids = np.array(market_ids)

print(f"Total sequences: {len(X)}")
print(f"Sequence shape: {X.shape}")
print(f"Target shape: {y.shape}")

Total sequences: 5890038
Sequence shape: (5890038, 20, 2)
Target shape: (5890038, 10)


In [6]:
unique_markets = np.unique(market_ids)
train_markets, test_markets = train_test_split(unique_markets, test_size = 0.2, random_state = 25)

train_mask = np.isin(market_ids, train_markets)
test_mask = np.isin(market_ids, test_markets)

X_train, y_train = X[train_mask], y[train_mask]
X_test, y_test = X[test_mask], y[test_mask]

print(f"Training sequences: {len(X_train)}")
print(f"Test sequences: {len(X_test)}")
print(f"Training markets: {len(train_markets)}")
print(f"Test markets: {len(test_markets)}")

Training sequences: 4673212
Test sequences: 1216826
Training markets: 94
Test markets: 24


In [7]:
n_train, lookback, n_features = X_train.shape
X_train_flat = X_train.reshape(-1, n_features)
X_test_flat = X_test.reshape(-1, n_features)

scaler_X = StandardScaler()
X_train_scaled = scaler_X.fit_transform(X_train_flat).reshape(n_train, lookback, n_features)
X_test_scaled = scaler_X.transform(X_test_flat).reshape(len(X_test), lookback, n_features)

scaler_y = StandardScaler()
y_train_scaled = scaler_y.fit_transform(y_train)
y_test_scaled = scaler_y.transform(y_test)

In [8]:
class PriceDataset(Dataset):
  def __init__(self, X, y):
    self.X = torch.FloatTensor(X)
    self.y = torch.FloatTensor(y)

  def __len__(self):
    return len(self.X)

  def __getitem__(self, idx):
    return self.X[idx], self.y[idx]

train_dataset = PriceDataset(X_train_scaled, y_train_scaled)
test_dataset = PriceDataset(X_test_scaled, y_test_scaled)

train_loader = DataLoader(train_dataset, batch_size = 64, shuffle = True)
test_loader = DataLoader(test_dataset, batch_size = 64, shuffle = False)

In [9]:
class LSTMForecaster(nn.Module):
  def __init__(self, input_size = 2, hidden_size = 64, num_layers = 2, forecast_horizon = 10):
    super().__init__()
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size, forecast_horizon)

  def forward(self, x):
    lstm_out, _ = self.lstm(x)
    last_hidden = lstm_out[:, -1, :]
    output = self.fc(last_hidden)
    return output

model = LSTMForecaster(input_size = 2, hidden_size = 64, num_layers = 2, forecast_horizon = FORECAST)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

num_epochs = 20
for epoch in range(num_epochs):
  model.train()
  total_loss = 0
  for batch_X, batch_y in train_loader:
    batch_X, batch_y = batch_X.to(device), batch_y.to(device)

    optimizer.zero_grad()
    outputs = model(batch_X)
    loss = criterion(outputs, batch_y)
    loss.backward()
    optimizer.step()

    total_loss += loss.item()

  avg_loss = total_loss / len(train_loader)
  if (epoch + 1) % 5 == 0:
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss: {avg_loss:.6f}")

Epoch [5/20], Loss: 0.001010
Epoch [10/20], Loss: 0.001002
Epoch [15/20], Loss: 0.000997
Epoch [20/20], Loss: 0.000985


In [11]:
model.eval()
all_preds = []
all_targets = []

with torch.no_grad():
  for batch_X, batch_y in test_loader:
    batch_X = batch_X.to(device)
    outputs = model(batch_X)

    preds = scaler_y.inverse_transform(outputs.cpu().numpy())
    targets = scaler_y.inverse_transform(batch_y.numpy())

    all_preds.append(preds)
    all_targets.append(targets)

all_preds = np.concatenate(all_preds, axis=0)
all_targets = np.concatenate(all_targets, axis=0)

preds_flat = all_preds.flatten()
targets_flat = all_targets.flatten()

mae = mean_absolute_error(targets_flat, preds_flat)
mse = mean_squared_error(targets_flat, preds_flat)
rmse = np.sqrt(mse)

print(f"MAE: {mae:.6f}")
print(f"MSE: {mse:.6f}")
print(f"RMSE: {rmse:.6f}")

MAE: 0.002265
MSE: 0.000132
RMSE: 0.011495
